<font color=pink, size=6><b>OmegAlpes Tutorial 1: Create optimisation model</b></font><br>
   
    In this tutorial, we will learn how to create optimisation model and simple power balance simulation in OmegAlpes. 

`Learning outcomes :`
>1. Model an energy system with OmegAlpes
2. General understanding of the model components, such as :
    - Time unit
    - Various type of energy units
    - Input data
    - Defining model objective
3. Processing optimisation result

    At the end of this tutorial, you will be able to find out how the result of John's PV installation.
    

# Creating Optimisation model 

    Let's start with creating an OmegAlpes model of John's home.
    
    He describes to us his home has several home appliances, with PV panels newly installed on top of his home.


<img src="./images/1-John house.JPG" width="550" height="150">
<center>Figure 1. John's house layout</center>

    He gives us his home electricity usage and pv production profile as shown in Figure 2.
    
<img src="./images/johns consumption.JPG" width="550" height="150" alt="Graph is produced using OmegAlpes">
<br>
<center>Figure 2. John's power profile</center>
    
    
    Let's start modelling !
    
    First we will import all the necessary functions needed to build our model.
    

In [ ]:
from Python_Scripts.Omegalpes_tutorial_1 import *

## Create time unit
    First step, we create timeunit.
    This module creates the Time object that defines the studied time period.
    

In [ ]:
time = TimeUnit(periods=24*1, dt=1)

    where :
    dt = delta t between values in hours (int or float), i.e. 1/6 will be 10 minutes. and 1 in this case will be 60 minutes
    period = 24*dt which means we will study the period of 24 hours
    
`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.time">time module</a>

## Create empty model
    Second step, we create an empty model where we will fill the optimization model.
    

In [ ]:
model = OptimisationModel(time=time, name='tutorial_1')

    time in this model will refer to the timeunit we set at first step
    
    
`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">model module</a>

## Create energy unit
    Third step, we create energy units which mainly consist of 3 major types/classes :
        1. Production units
        2. Consumption units
        3. Storage units
    In this level, we will learn how to utilize the first two.
    
    These classes have multiple variety of classes, some of the commonly used are:
        1. FixedEnergyUnit: energy unit with a fixed power profile
        2. VariableEnergyUnit: energy unit with a variable power profile
    We will learn how to create these units in this tutorial

`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.energy_units">energy units module</a>

### Create fixed energy units
    We will start with fixed energy units with fixed power profile data we got from John.
    
    First, we start with consumption units. As its name implies, this energy units have power flow direction always ‘in’.
    
    We will import the home consumptions data from the csv file provided.

In [ ]:
consumption_profile = open("data/house tot con 24 kwh.csv", "r")

house_cons_file = [p for p in map(float, consumption_profile)]

    Data from the file will be stored in "house_cons_file" with map function. Keyword: "creating a list of float values from string values" to learn more.
    
    Then, we create the consumption unit.
    
    We define it as FixedConsumptionUnit since it has fixed load profile.

In [ ]:
house_consumption = FixedConsumptionUnit(time, 'house_consumption', p=house_cons_file, energy_type=elec)

    This consumption unit has attributes :
    - time which refers to timeunit
    - label as "house_consumption" which will be shown in the plot result
    - p (instantaneous power demand) which refers to the fixed load profile
    - energy_type = electrical; this is important to define when we have multiple energy types such as electrical and heat

***
    Next, we create production units. This energy units have power flow direction always ‘out’.
    
    We will import the pv production data from the csv file provided   

In [ ]:
pv_profile = open("data/pv prod 24 kwh.csv", "r")

pv_production_daily = [p for p in map(float, pv_profile)]

      
    Then, we create the production unit.
    
    We define it as FixedProductionUnit since it has fixed production profile.

In [ ]:
pv_production = FixedProductionUnit(time, name='pv_production', p=pv_production_daily ,energy_type=elec)

    This production unit has attributes similar to consumption units :
    - time always refer to timeunit
    - label as "pv_production"
    - p = instantaneous power production known by advance loaded from the csv file reference
    - energy_type = electrical; which is important to define
    
    Now, we have created 2 fixed energy units
    
<img src="./images/1-fixed_units.JPG" width="550" height="150">
<center>Figure 3. Adding fixed units to model</center>

### Create variable energy units
    After creating fixed energy units, we will create variable energy units. VariableEnergyUnit is an energy unit with a va=riable power profile following the constraints we set and optimization result.
    
    We create VariableProductionUnit and VariableConsumptionUnit, which allow a variation of power between p_min and p_max.

In [ ]:
grid_import = VariableProductionUnit(time, 'grid_import', energy_type=elec, p_min=0)

grid_export = VariableConsumptionUnit(time, 'grid_export', energy_type=elec, p_min=0)

    This VariableProductionUnit and VariableConsumptionUnit will represent electricity import and export respectively.
    
    These variable energy units have attributes:
    - time refers to timeunit as always
    - labels for result
    - energy_type = electrical; which is important to define
    - p_min (minimal instantaneous power production) = 0, so that it won't go to negative value
    - p_max (maximal instantaneous power production) is not defined, so there is no maximal value restriction
    
    Now, we have added another 2 variable energy units
<img src="./images/1-variable_units.JPG" width="550" height="150">
<center>Figure 4. Adding variable units to model</center>
<br>



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.production_units">production units module</a> & <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.consumption_units">consumption units module</a>

## Review
    Good job! We have created all the necessary components for our energy model.
    Let us do some reviewing. Complete this challenge by running cell below :

In [ ]:
%run Python_Scripts/quiz/quiz_tutorial11

## Define model objective
    Fourth step, we define the model objective   

In [ ]:
grid_import.minimize_production()

    OmegAlpes will give the optimization result based on the objectives, which in this case is to minimize production from  grid import (Variable Production Unit).

`learn more optimization objective functions for energy units:`<a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.production_units">production units module</a> & <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.consumption_units"> consumption units module</a>

## Create energy nodes
    Fifth step, we create energy nodes to connect all the energy units we created.
    This nodes handles energy transmission between production, consumption, conversion and storage connected to it.

In [ ]:
elec_node = EnergyNode(time, name="electrical_node", energy_type=elec)

    These energy nodes have attributes:
    - time refers to timeunit as always
    - name label
    - energy_type = electrical; which is important to define
    
    Then, we connect energy units to the energy node we create

In [ ]:
elec_node.connect_units(grid_import, grid_export, pv_production,house_consumption)

    Great! Now, we have created our energy system
<img src="./images/1-elec_node.JPG" width="550" height="150">
<center>Figure 5. Adding electrical node to model</center>

    As you can see, production units have power flow direction in’ 



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.energy_nodes">energy nodes module</a>

## Add the energy nodes to the optimisation model
    Sixth step, we add the energy system (nodes and all connected units) to the model we defined at second step
    Check that the timeunit is the same for the model and all the units

In [ ]:
model.add_nodes(elec_node)



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">optimisation model</a>

## Launch the optimisation
    Seventh step, we launch the optimisation for the model we created.
    This will solve the optimization model and updates all variables values.

In [ ]:
model.solve_and_update()



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">optimisation model</a>

## Plot result
    Eighth step, we plot the result into graph.

In [ ]:
plot_node_energetic_flows(elec_node)

    The argument we enter is the node we defined at fifth step.
    
    plot_node_energetic_flows will plot the energy flows that go through an EnergyNode

`learn more plotting functions:` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.utils.plots">plots module</a>

     Let us compare it with the data we got
<img src="./images/1-result.jpg" alt="Graph is produced using OmegAlpes">
<center>Figure 6. Comparison of John's power profile before and after the optimisation</center>

    As we can observe in these graphs, OmegAlpes will import when there is a need for electricity and export when there is  surplus. Here is another quiz!

In [ ]:
%run Python_Scripts/quiz/quiz_tutorial13

## Result processing
    There are also some way to process the result aside from plotting.
    
    For example, if we want to show total house consumption. We can run following line

In [ ]:
print('House consumption = {0} kWh'.format(sum(house_consumption.p.get_value())))
    

    It is time for another challenge! Run this following cell :

In [ ]:
%run Python_Scripts/quiz/quiz_tutorial12

In [ ]:
#work your challenge here

## Discussion
    Congratulation for making it this far. Let's see how good you are! Run the cell to see if you can you clear all the following challenges?

In [ ]:
%run Python_Scripts/quiz/quiz_tutorial16

***
    Want hint of the correct answers ? Try inserting magic sentence from the words you collected in this tutorial in below  challenge!

In [ ]:
%run Python_Scripts/quiz/quiz_tutorial14

***
    Let's review what you have learnt so far !
    
`Skills you have acquired:`
>- [x] Model a simple energy system with OmegAlpes
- [x] General understanding of basic model components, such as :
- [x] Simple optimisation simulation
    
    As you may have realized after observing result plot, installing PV alone is not the optimal solution. Let's help John   further with better solutions, such as energy storage. Of course, as an energy consultant you need to convince him with numbers. Thus, let's utilize the capabilities of OmegAlpes to show John how good our other solutions are. Level up your current skills so that you are able to simulate these solutions with OmegAlpes.
    
    All of these new skills will be covered in level 2 tutorial. You will learn how to add more components and constraints  to the model for better result. You may expect some solutions such as adding storage unit, comparing costs and utilizing thermal energy. Therefore, let's return to the dashboard to access these other tutorials!
     